# **Decision Tree and Random Forest**

1. Basic Part : Implement a **Decision Tree** model and predict whether the patients in the validation set have diabetes
> * Step 1 : Load the input data
> * Step 2 : Calculate the Entropy and Information Gain
> * Step 3 : Find the Best Split
> * Step 4 : Split into 2 branches
> * Step 5 : Build decision tree
> * Step 6 : Split data into training set and validation set
> * Step 7 : Train a decision tree model with training set
> * Step 8 : Predict the cases in the *validation set* by using the model trained in *Step7*
> * Step 9 : Calculate the f1-score of your predictions in *Step8*

2. Advanced Part : Build a **Random Forest** model to make predictions
> * Step 1 : Load the input data
> * Step 2 : Load the test data
> * Step 3 : Build a random forest
> * Step 4 : Predict the cases in the test data by using the model trained in *Step3*

# **Basic Part**

Implement a Decision Tree model by completing the following functions.

## Import Packages

In [1]:
import numpy as np
import pandas as pd
import math
import random
from numpy import sqrt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

## Step1: Load the input data

First, load the input file **input_basic.csv**

In [2]:
input_data = pd.read_csv('input_basic.csv')
input_data

,age,bmi,gender,height,weight,glucose_apache,heart_rate_apache,resprate_apache,sodium_apache,diabetes_mellitus
0,70.0,25.984659,1,172.7,77.50,116.0,101.0,49.0,137.0,0
1,30.0,31.310368,1,170.2,90.70,71.0,39.0,33.0,144.0,0
2,54.0,24.388824,1,177.8,77.10,120.0,120.0,31.0,141.0,0
3,65.0,34.141074,0,170.2,98.90,73.0,48.0,36.0,140.0,1
4,49.0,22.564743,1,172.7,67.30,207.0,119.0,6.0,144.0,0
5,62.0,29.424010,0,154.9,70.60,113.0,60.0,32.0,137.0,0
6,85.0,27.673574,1,154.9,66.40,102.0,49.0,36.0,142.0,0
7,65.0,22.269432,1,177.8,70.40,333.0,59.0,6.0,145.0,1
8,85.0,35.879362,0,165.1,97.80,124.0,92.0,30.0,136.0,0
9,81.0,20.859375,0,160.0,53.40,136.0,118.0,52.0,138.0,0


## Global attributes

Define the global attributes

In [3]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = input_data.shape[1] - 1

In [4]:
mode = "basic"

## Step2 : Calculate the Entropy and Information Gain

Calculate the information gain and entropy values before separate data into left subtree and right subtree

In [ ]:
from logging import log
def entropy(data):
  """
  This function measures the amount of uncertainty in a probability distribution
  args: 
  * data(type: DataFrame): the data you're calculating for the entropy
  return:
  * entropy_value(type: float): the data's entropy
  """
  
  if data.empty:
    return 0

  # Count values
  p = list(data["diabetes_mellitus"]).count(1)
  n = list(data["diabetes_mellitus"]).count(0)
  
  p_positive = p/(p+n)
  p_negative = n/(p+n)
  if p_positive == 0 or p_negative == 0:
    entropy_value = 0
  else:
    entropy_value = -p_positive*math.log2(p_positive) - p_negative*math.log2(p_negative)
  
  return entropy_value

In [ ]:
def information_gain(data, mask):
  """
  This function will calculate the information gain
  args:
  * data(type: DataFrame): the data you're calculating for the information gain
  * mask(type: Series): partition information(left/right) of current input data, 
    - boolean 1(True) represents split to left subtree
    - boolean 0(False) represents split to right subtree
  return:
  * ig(type: float): the information gain you can obtain by classify data with this given mask
  """

  left = data[mask]
  right = data[~mask]
  
  w_left = len(left) / (len(left) + len(right))
  w_right = len(right) / (len(left) + len(right))
  ig = entropy(data) - (w_left*entropy(left) + w_right*entropy(right))

  return ig

## Step3 : Find the Best Split

Find the best split combination, **feature** and **threshold**, by calculating the information gain

In [ ]:
def find_best_split(data):
  """
  This function will find the best split combination of data
  args:
  * data(type: DataFrame): the input data
  return
  * best_ig(type: float): the best information gain you obtain
  * best_threshold(type: float): the value that splits data into 2 branches
  * best_feature(type: string): the feature that splits data into 2 branches
  """

  best_ig = 0
  best_threshold = 0
  best_feature = ""
  for feature in data.loc[:, data.columns != "diabetes_mellitus"]:
    temp_data = data.sort_values(feature, ignore_index=True)
    unique_values = temp_data[feature].unique()
    prev = None
    for i in unique_values:
      if prev != None:
        midpoint = (prev + i) / 2
        mask = (temp_data[feature] <= midpoint)
        ig = information_gain(temp_data, mask)
        if ig > best_ig:
          best_ig = ig
          best_threshold = midpoint
          best_feature = feature
      prev = i
  
  return best_ig, best_threshold, best_feature

## Step4 : Split into 2 branches

Using the best split combination you find in function *find_best_split()* to split data into Left Subtree and Right Subtree

In [ ]:
def make_partition(data, feature, threshold):
  """
  This function will split the data into 2 branches
  args:
  * data(type: DataFrame): the input data
  * feature(type: string): the attribute(column name)
  * threshold(type: float): the threshold for splitting the data
  return:
  * left(type: DataFrame): the divided data that matches(less than or equal to) the assigned feature's threshold
  * right(type: DataFrame): the divided data that doesn't match the assigned feature's threshold
  """
  
  left = data.loc[data[feature] <= threshold]
  right = data.loc[data[feature] > threshold]
  
  return left, right

## Step5 : Build Decision Tree
Use the above functions to implement the decision tree

Instructions: 
1.  If current depth < max_depth and the remaining number of samples > min_samples_split: continue to classify those samples
2.  Use function *find_best_split()* to find the best split combination
3.  If the obtained information gain is **greater than 0**: can build a deeper decision tree (add depth)
4. Use function *make_partition()* to split the data into two parts
5. Save the features and corresponding thresholds (starting from the root) used by the decision tree into *ans_features[]* and *ans_thresholds[]* respectively




In [9]:
def build_tree(data, max_depth, min_samples_split, depth):
  """
  This function will build the decision tree
  args:
  * data(type: DataFrame): the data you want to apply to the decision tree
  * max_depth: the maximum depth of a decision tree
  * min_samples_split: the minimum number of instances required to do partition
  * depth: the height of the current decision tree
  return:
  * subtree: the decision tree structure including root, branch, and leaf (with the attributes and thresholds)
  """

  temp_data = data.copy()
  if mode == "advanced":
    temp = temp_data["diabetes_mellitus"]
    temp_data = temp_data.drop(["diabetes_mellitus"], axis = 1)
    temp_data = temp_data.sample(n = n_features, axis = 1, replace = False)
    temp_data["diabetes_mellitus"] = temp

  # check the condition of current depth and the remaining number of samples
  if depth < max_depth and len(data) > min_samples_split:

    # call find_best_split() to find the best combination
    ig, threshold, feature = find_best_split(temp_data)

    # check the value of information gain is greater than 0 or not 
    if ig > 0 :
      # update the depth
      depth = depth + 1

      # call make_partition() to split the data into two parts
      left, right = make_partition(data, feature, threshold)

      # If there is no data split to the left tree OR no data split to the left tree
      if len(left) == 0 or len(right) == 0:

        # return the label of the majority
        p = list(data["diabetes_mellitus"]).count(1)
        n = list(data["diabetes_mellitus"]).count(0)
        label = 1 if p >= n else 0
        return label
      else:
        question = "{} {} {}".format(feature, "<=", threshold)
        subtree = {question: []}

        # call function build_tree() to recursively build the left subtree and right subtree
        left_subtree = build_tree(left, max_depth, min_samples_split, depth)
        right_subtree = build_tree(right, max_depth, min_samples_split, depth)

        if left_subtree == right_subtree:
          subtree = left_subtree
        else:
          subtree[question].append(left_subtree)
          subtree[question].append(right_subtree)
    else:
      # return the label of the majority
      p = list(data["diabetes_mellitus"]).count(1)
      n = list(data["diabetes_mellitus"]).count(0)
      label = 1 if p >= n else 0
      return label
  else:
    # return the label of the majority
    p = list(data["diabetes_mellitus"]).count(1)
    n = list(data["diabetes_mellitus"]).count(0)
    label = 1 if p >= n else 0
    return label

  return subtree

An example of the output from *build_tree()* 
```
{'bmi <= 33.5': [1, {'age <= 68.5': [0, 1]}]}
```
Therefore, 
```
ans_features = ['bmi', 'age']
ans_thresholds = [33.5, 68.5]
```

In [10]:
decisionTree = build_tree(input_data, max_depth, min_samples_split, depth)
decisionTree

{'glucose_apache <= 235.5': [{'heart_rate_apache <= 143.5': [0, 1]}, 1]}

## Step6 : Split data

Split data into training set and validation set

In [14]:
num_train = 20
num_validation = 10

training_data = input_data.iloc[:num_train]
validation_data = input_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(input_data.shape)
print(training_data.shape)
print(validation_data.shape)

(30, 10)
(20, 10)
(10, 10)


## Step7 to Step9 : Make predictions with a decision tree

Define the attributions of the decision tree

In [15]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = x_train.shape[1]

In [16]:
def classify_data(instance, tree):
  """
  This function will predict/classify the input instance
  args:
  * instance: a instance(case) to be predicted
  return:
  * answer: the prediction result (the classification result)
  """

  equation = list(tree.keys())[0] 
  if equation.split()[1] == '<=':
    temp_feature = equation.split()[0]
    temp_threshold = equation.split()[2]
    if instance[temp_feature] > float(temp_threshold):
      answer = tree[equation][1]
    else:
      answer = tree[equation][0]
  else:
    if instance[equation.split()[0]] in (equation.split()[2]):
      answer = tree[equation][0]
    else:
      answer = tree[equation][1]

  if not isinstance(answer, dict):
    return answer
  else:
    return classify_data(instance, answer)


def make_prediction(tree, data):
  """
  This function will use your pre-trained decision tree to predict the labels of all instances in data
  args:
  * tree: the decision tree
  * data: the data to predict
  return:
  * y_prediction: the predictions
  """
  
  # [Note] You can call the function classify_data() to predict the label of each instance
  y_prediction = []
  for i in range(len(data)):
    y_prediction.append(classify_data(data.iloc[i], tree))

  return y_prediction


def calculate_score(y_true, y_pred):
  """
  This function will calculate the f1-score of the predictions
  args:
  * y_true: the ground truth
  * y_pred: the predictions
  return:
  * score: the f1-score
  """
  
  score = f1_score(y_true, y_pred)
  
  return score

In [ ]:
decision_tree = build_tree(training_data, max_depth, min_samples_split, depth)

y_pred = make_prediction(decision_tree, x_validation)

# **Advanced Part**

## Step1: Load the input data

First, load the input file **input_advanced.csv**

In [19]:
advanced_data = pd.read_csv('input_advanced.csv')

You can split *advanced_data* into training set and validation set

In [20]:
num_train = round(len(advanced_data)*0.75)
num_validation = round(len(advanced_data)*0.25)

training_data = advanced_data.iloc[:num_train]
validation_data = advanced_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

## Step2 : Load the test data
Load the input file **input_test.csv** to make predictions with the pre-trained random forest model

In [21]:
x_test = pd.read_csv('input_test.csv')
x_test

,age,bmi,gender,height,weight,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,...,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
0,62,32.866392,1,177.80,103.9,1,31.0,10.30,4,6,...,36.4,0,157,26,134,36.1,0,4.56,0.06,0.03
1,82,23.582766,0,157.50,58.5,0,26.0,0.54,3,4,...,32.8,0,42,25,142,36.1,0,6.00,0.14,0.06
2,61,31.684520,1,172.70,94.5,0,16.0,1.11,4,6,...,35.3,0,129,6,131,36.8,0,8.59,0.05,0.03
3,58,45.156250,0,160.00,115.6,0,19.0,0.70,1,4,...,30.1,1,131,23,138,34.9,1,16.03,0.33,0.22
4,74,25.817016,1,172.70,77.0,0,25.0,0.93,4,6,...,34.5,0,55,12,135,36.3,0,45.80,0.12,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,73,17.943584,0,157.48,44.5,0,12.0,0.30,4,6,...,33.8,0,129,9,144,36.9,0,7.70,0.02,0.01
836,79,29.049732,1,167.60,81.6,0,48.0,2.19,4,6,...,42.7,0,163,9,139,36.4,0,10.77,0.06,0.03
837,85,24.627827,0,152.40,57.2,0,11.0,0.48,3,5,...,29.5,0,67,9,139,36.6,0,7.35,0.16,0.05
838,68,32.510940,1,193.00,121.1,0,14.0,0.64,4,6,...,37.5,0,61,10,140,36.9,1,16.02,0.00,0.00


## Step3 : Build a Random Forest

In [30]:
# Define the attributes

max_depth = 7
depth = 0
min_samples_split = 4

# total number of trees in a random forest
n_trees = 250

# number of features to train a decision tree
n_features = 5

# the ratio to select the number of instances
sample_size = 0.6
n_samples = int(training_data.shape[0] * sample_size)

mode = "advanced"

In [31]:
def build_forest(data, n_trees, n_features, n_samples):
  """
  This function will build a random forest.
  args:
  * data: all data that can be used to train a random forest
  * n_trees: total number of tree
  * n_features: number of features
  * n_samples: number of instances
  return:
  * forest: a random forest with 'n_trees' of decision tree
  """

  forest = []
  for i in range(n_trees):

    bootstrap_sample = data.sample(n = n_samples, replace = True)

    tree = build_tree(bootstrap_sample, max_depth, min_samples_split, depth)
    forest.append(tree)

  return forest

In [32]:
forest = build_forest(training_data, n_trees, n_features, n_samples)

## Step4 : Make predictions with the random forest

In [36]:
def majority_vote(values):
  p = values.count(1)
  n = values.count(0)
  return 1 if p >= n else 0

def make_prediction_forest(forest, data):
  """
  This function will use the pre-trained random forest to make the predictions
  args:
  * forest: the random forest
  * data: the data used to predict
  return:
  * y_prediction: the predicted results
  """

  idx = 1
  tree_results = []
  for tree in forest:
    results = []
    for i in range(len(data)):
      results.append(classify_data(data.iloc[i], tree))
    tree_results.append(results)
    if data_type == "validation":
      print("f1-score of tree", idx, "=", calculate_score(y_validation, results))
    idx += 1
    
  y_prediction = []
  for results in zip(*tree_results):
    label = majority_vote(results)
    y_prediction.append(label)
  
  if data_type == "validation":
    return calculate_score(y_validation, y_prediction)

  return y_prediction

In [37]:
data_type = "validation"
print("Final f1-score =", make_prediction_forest(forest, x_validation))
data_type = "test"
y_pred_test = make_prediction_forest(forest, x_test)

f1-score of tree 1 = 0.651186790505676
f1-score of tree 2 = 0.654282765737874
f1-score of tree 3 = 0.678627968337731
f1-score of tree 4 = 0.6504653567735265
f1-score of tree 5 = 0.6338329764453962
f1-score of tree 6 = 0.6363131593559133
f1-score of tree 7 = 0.669710806697108
f1-score of tree 8 = 0.6342003320420586
f1-score of tree 9 = 0.685404339250493
f1-score of tree 10 = 0.6070252469813392
f1-score of tree 11 = 0.6912350597609561
f1-score of tree 12 = 0.612534059945504
f1-score of tree 13 = 0.669367088607595
f1-score of tree 14 = 0.6688034188034188
f1-score of tree 15 = 0.6492039034411915
f1-score of tree 16 = 0.6506614404703576
f1-score of tree 17 = 0.5756373937677054
f1-score of tree 18 = 0.64390756302521
f1-score of tree 19 = 0.6537842190016104
f1-score of tree 20 = 0.6656519533231862
f1-score of tree 21 = 0.6016172506738544
f1-score of tree 22 = 0.6316377864728898
f1-score of tree 23 = 0.665631469979296
f1-score of tree 24 = 0.6556224899598394
f1-score of tree 25 = 0.67663744198